In [1]:
# ici, il importe les libraries commun de travail

import datetime
from csv import reader
from datetime import datetime
from math import exp
from random import random
from random import randrange

import numpy as np


# ici il prépare des fonctions standards qui vont ètre utisés fréquemment le long du projet



# ici une fonction qui catégorise un attribut dont les valeurs sont pris dans la liste l 
def from_values_to_categorical(l):
    """
  Convert a list of values to a list of categorical value
  l : list to consider(list)
  return : a list of categorical value (list of int)
  """
    dictionnary = from_list_to_dic(l) # fonction qui donne un dictionnaire: la key: l'élement l et la valeur : c'est son index dans la liste
    categorical = [dictionnary[key] for key in l] # la liste des indices des élements de l qui vont présenter les catégories
    return categorical, dictionnary


# ici la fonction qui donne un dictionnaire à partir d'une liste:
# la key: c'est l'élement de l 
# la valeur : c'est son index dans la liste
def from_list_to_dic(l):
    """
  Creating a dictionnary  from a list of values
  l : list of value
  return : a dictionnaire associating a unique number to each value
  """
    l = list(set(l))
    n = len(l)
    dictionnary = {}
    for i in range(n):
        dictionnary[l[i]] = i # voilà, ici il associe à chaque key (l[i]) sa valeur qui est i (son index)
    return dictionnary


# ici cette fonction convertit des dates en des nombres car les machines learning modèles ne traite que 
# des données numériques :
# comment est cette convertion? pour chaque date, on calcule la période qui la sépare de 01-01-2000
# Aprés, pour chaque période calculée, on calcule sa période qui la sépare du minimum de ces périodes.
# donc, on obtient d'une liste des dates , des listes des nombres 
def from_date_to_number(l, dateFormatter):
    """
  Convert a list of date to a list of float
  l : list to consider (list of str)
  dateFormatter : Format of the date (str)
  return : a list of float (list of float)
  """
    # l: liste contenant des chaines de caractères représentant la date
    list_datev1 = []
    for date in l:
        date2 = datetime.strptime(date, dateFormatter) 
        #strptime: fonction de la librarie datetime qui prend la date sous forme string ainsi que le format de 
        # la date voulu et le converti en datetime object 
        date1 = (date2 - datetime(2000, 1, 1)).total_seconds()
        #datetime.total_seconds(): calcule une différence des dates en secondes 
        # donc ici il calcule la période entre les dates à traiter et la date(01-01-2000) en secondes 
        list_datev1.append(date1)

    min_date = min(list_datev1) # ici on prends la période minimale

    list_date = [(date - min_date) for date in list_datev1]# liste des différences entre les différents périodes  
    # calculées précedemment et la periode minimale
    return list_date


# Convertion rate for the currency 
# a 7*7 matrix containing currency rate 
currency_rate = np.array([[1, 0.85, 1.18, 1.08, 10.20, 10.49, 1.49], [1.17, 1, 1.39, 1.27, 11.97, 12.33, 1.74],
                          [0.85, 0.72, 1, 0.91, 8.63, 8.88, 1.26], [0.93, 0.79, 1.09, 1, 9.44, 9.71, 1.37],
                          [0.098, 0.083, 0.12, 0.11, 1, 1.03, 0.15], [0.095, 0.081, 0.11, 0.1, 0.97, 1, 0.14],
                          [0.67, 0.57, 0.8, 0.73, 6.86, 7.07, 1]])



# ici , on convertit la currency de la valeur des tickets en utilsant la matrice précedente
def conversion_highticket(data):
    """
  Apply the conversion rate to the dataframe
  """
    if isinstance(data[6], (int, float)): # boolean function : check if data[6] is an integer or a float
        currency1 = min(int(data[6]), 7)
        currency2 = 1
        amount = data[3] # 3rd column contient les prix highticket
    return (amount * currency_rate[currency1 - 1][currency2 - 1])


def conversion_lowticket(data):
    """
  Apply the conversion rate thedataframe
  """
    if isinstance(data[6], (int, float)): # boolean function : check if data[6] is an integer or a float
        currency1 = min(int(data[6]), 7)
        currency2 = 1
        amount = data[2]# 2nd column contient les prix lowticket
    return (amount * currency_rate[currency1 - 1][currency2 - 1])


def conversion_lowinvest(data):
    """
  Apply the conversion rate thedataframe
  """
    if isinstance(data[6], (int, float)):
        currency1 = min(int(data[6]), 7)
        currency2 = 1
        amount = data[4]# 4rd column contient les prix lowinvest
    return (amount * currency_rate[currency1 - 1][currency2 - 1])


def conversion_highinvest(data):
    """
  Apply the conversion rate thedataframe
  """
    if isinstance(data[6], (int, float)):
        currency1 = min(int(data[6]), 7)
        currency2 = 1
        amount = data[5]# 3rd column contient les prix highinvest
    return amount * currency_rate[currency1 - 1][currency2 - 1]

# élimine les colonnes et les lignes en posant un certain threshold à partir duquel , il élimine 
# partie coding ici est claire
def take_off_missing_value(data, column, percent, row):
    """
  Take off the missing value from a panda dataframe
  data : panda dataframe
  columns : if we want to take of the columns with a certain percentage of value missing (boolean)
  percent : percentage of value from which we take of the column(float between 0 and 100)
  row : if we want to take of the rows with a certain percentage of value missing (boolean)
  """
    column_with_nan = data.columns[data.isnull().any()]

    # Taking off the column with more 30% of missing
    if column:
        for column1 in column_with_nan:
            if data[column1].isnull().sum() * 100.0 / data.shape[0] > percent:
                data.drop(column1, 1, inplace=True)

    # Taking off the row with missing values
    if row:
        index_with_nan = data.index[data.isnull().any(axis=1)]
        data.drop(index_with_nan, 0, inplace=True)
    return data


# il y a des valeurs manquantes dans la colonnes 'exist_date'
# on les remplace par 31-12-9999 22:00:00
def replacemissingexitsdate(x):
    '''
    Replace a missing exit date by ' 9999-12-31 22:00:00 '
    :param x: a date (str)
    :return: x if it's not empty, ' 9999-12-31 22:00:00 ' otherwise
    '''
    if x[1] == ' ':
        return ' 9999-12-31 22:00:00 '
    else:
        return x

    
# pareil pour la colonne'entry_date'
def replacemissingentrysdate(x):
    '''
    Replace a missing entry date by ' 0001-01-01 01:01:01 '
    :param x: a date (str)
    :return: x if it's not empty, ' 0001-01-01 01:01:01 ' otherwise
    '''
    if x[1] == ' ':
        return ' 0001-01-01 01:01:01 '
    else:
        return x

# replace all the missing values in id ('Nan value') by a 0
def replace_missing_id(x):
    '''
    Replace a missing
    :param x: the id
    :return: x if it's not empty, 0 otherwise
    '''
    if type(x) == str: # pour dire qu'il s'agit d'un Nan value sous forme str
        return 0
    else:
        return x


# Load a CSV file but put it in a list of list instead of a dataframe object
# on sait pas pourquoi mais il a fait comme ça ici en fait.
def load_csv(filename):
    '''
    Load a csv and put it in a list of list
    :param filename: name of the csv file (str)
    :return: a dataset (list of list)
    '''
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


# Convert string column to float
# pour les colonnes à valeurs str , on les change en des valeurs numériques
def str_column_to_float(dataset, column):
    '''

    :param dataset: dataset to modify (list of list)
    :param column: the name of the column (list of str)
    :return: the same dataset with the name of the column converted into float
    '''
    for row in dataset:
        row[column] = float(row[column].strip())


# Convert string column to integer
#pareil au précedent
def str_column_to_int(dataset, column):
    '''
    Convert string column into integer
    :param dataset: dataset to modify (list of list)
    :param column: the name of the column (list of str)
    :return: the same dataset with the name of the column converted into int
    '''
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup


# Find the min and max values for each column
# chercher le smin et max de chaque colonnes et mettre chaque couple dans une liste et tous ces listes 
# dans une grande liste
def dataset_minmax(dataset):
    '''
    Find the min and max values for each column
    :param dataset: dataset to consider (list of list)
    :return: minimum and maximum value in the dataset (list of float)
    '''
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats


# Rescale dataset columns to the range 0-1
# il utilise la technique min_max scaling (connu en statistiques)
def normalize_dataset(dataset, minmax):
    '''
    Rescale dataset columns to the range 0-1
    :param dataset: dataset to consider (list of list)
    :param minmax: minimum and maximum value in the dataset (list of float)
    :return: normalize dataset (list of list)
    '''
    for row in dataset:
        for i in range(len(row) - 1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0]) # formule de min-max scaling


# Split a dataset into k folds
# pour utiliser les cross validation scoring method, on doit diviser le dataset en n folds
# aprés utiliser chacune comme une training set et tous les autres come une validation set
# continuer à faire ça en parcourant tous les n folds et calculer le score d'accuracy à chaque fois
# puis donner le score moyen d'accuracy
def cross_validation_split(dataset, n_folds):
    '''

    :param dataset: dataset to consider (list of list)
    :param n_folds: number of folds to split the dataset in (int)
    :return: dataset splitted into k folds (list of list of list)
    '''
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


# Calculate accuracy percentage
# coding littéral de l'acuuracy metric 
def accuracy_metric(actual, predicted):
    '''
    Calculate accuracy percentage
    :param actual: the actual value (list of float)
    :param predicted: the value predicted by the model (list of float)
    :return: the accuracy percentage (float)
    '''
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0 # ça donne l'accuracy score


#sensitivity accuracy
def sensitivity_metric(predicted, actual):
    '''
    Calculate sensitivity percentage
    :param predicted: the value predicted by the model (list of float)
    :param actual: the actual value (list of float)
    :return: the sensitivity percentage (float)
    '''
    n_total = len(predicted)
    n = 0
    cpt = 0
    for i in range(n_total):
        if actual[i] == 1:
            n += 1
            if predicted[i] == 1:
                cpt += 1
    return cpt / float(len(actual)) * 100.0

# voilà il utilise la technique de cross validation score : 
# Split a dataset into k folds
# pour utiliser le cross validation scoring method, on doit diviser le dataset en n folds
# aprés utiliser chacune comme une training set et tous les autres come une validation set
# continuer à faire ça en parcourant tous les n folds et calculer le score d'accuracy à chaque fois
# puis donner le score moyen d'accuracy
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    '''
    Evaluate the performance of the algorithm over the different fold
    :param dataset: dataset to consider (list of list)
    :param algorithm: our machine learning model
    :param n_folds: number of folds to split the data into (int)
    :param args:
    :return: the list of score of the model (list of float)
    '''
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        sensitivity = sensitivity_metric(predicted, actual)
        scores.append(sensitivity)
    return scores


# Calculate neuron activation for an input
# weighted somme (somme des inputs pondérés chacune par son poids(weight_vector w))
def activate(weights, inputs):
    '''
    Calculate neuron activation for an input
    :param weights: list of weights (list of float)
    :param inputs: list of inputs (list of float)
    :return: the value of the activation (float)
    '''
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation


# Transfer neuron activation
# on utilise ici le sigmoid activation function
def transfer(activation):
    '''
    Compute the activation function of the neuron
    :param activation: the value of the neuron activation (float)
    :return: value of the transfer function (float)
    '''
    return 1.0 / (1.0 + exp(-activation)) # sigmoid function


# Forward propagate input to a network output
# on code littéralement le technique de forward propagation through neural network's layers
def forward_propagate(network, row):
    """
    Forward propagate input to a network output
    :param network: our network
    :param row: the input (list)
    :return: our updated input
    """
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


# Calculate the derivative of an neuron output
def transfer_derivative(output):
    """
    Calculate the derivative of an neuron output
    :param output: a neuron output (float)
    :return: derivative the neuron output float
    """
    return output * (1.0 - output) 


# Backpropagate error and store in neurons
# ici on commence à coder le backpropagation technique (on enregistre les erreurs de forward propagation
# dans les neurones en commençant du dernière couche jusqu'au première couche)
def backward_propagate_error(network, expected):
    """

    :param network: our neural network (list)
    :param expected: the list of the expected value (list of float)
    :return:
    """
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])


# Backpropagate error and store in neurons
# ça se répète ici : la mème fonction que prédemment
def backward_propagate_error(network, expected):
    """
    Backpropagate error and store in neurons
    :param network: our neural network (list)
    :param expected: a list of expected value (list of float)
    :return:
    """
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])


# Update network weights with error 
def update_weights(network, row, l_rate):
    '''
    Update network weights with error
    :param network: our neural network (list of float)
    :param row:
    :param l_rate: the learning rate (float)
    :return:
    '''
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# training the neural network function 
#étape 1: forward propagation 
#etape2: back propagation avec mise à jour des weights
def train_network(network, train, l_rate, n_epoch, n_outputs):
    """
    Train our neural network
    :param network: our neural network (list of float)
    :param train:
    :param l_rate: our learning rate (float)
    :param n_epoch: the number of epoch for the training (int)
    :param n_outputs: the expected number of output (int)
    :return:
    """
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)


# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    '''
    Initialize a network
    :param n_inputs: the number of inputs (int)
    :param n_hidden: the number of hidden layer (int)
    :param n_outputs: the number of output (int)
    :return: a neural network with random weights (list of of dictionary)
    '''
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network


# Make a prediction with a network
def predict(network, row):
    '''
    Make a prediction with a network
    :param network: our neural network (list of dictionary)
    :param row:
    :return: prediction of the neural network according the network
    '''
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# function qui code littéralement la technique de back propagation
def back_propagation(train, test, l_rate, n_epoch, n_hidden, list_network):
    """
    Perform the backpropagation in our neural network
    :param train: our training dataset (list of list of float)
    :param test: our test dataset (list of list of float)
    :param l_rate: our learning rate (float)
    :param n_epoch: the number of epoch of the training period (int)
    :param n_hidden: the number of hidden layer (int)
    :param list_network: a list of neural network (list of list of dict)
    :return: the prediction made by the neural network (list of float)
    """
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    list_network.append(network)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return predictions

# scoring via le cross validation technique 
def find_best_mode(dataset, back_propagation, n_folds):
    """
    Find the best feed forward neural for a given dataset.
    Basically a fine tuning
    :param dataset: our dataset (list of float)
    :param back_propagation:
    :param n_folds: the number of folds for the k fold validation
    :return: the best model for our dataset(list of dict) and its sensitivity(float)
    """
    list_rate = [0.001, 0.01, 0.1, 0.2, 0.5, 0.7, 1]
    list_hidden = [1, 5, 10, 25, 50, 100, 200, 500, 1000]
    list_epoch = [1, 5, 10, 25, 100, 200, 500, 1000]
    max_sensitivity = 0
    for l_rate in list_rate:
        for n_hidden in list_hidden:
            for n_epoch in list_epoch:
                list_network = []
                scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden, [])
                if (sum(scores) / float(len(scores))) > max_sensitivity:
                    max_sensitivity = (sum(scores) / float(len(scores)))
                    if len(list_network) > 0:
                        model = list_network[(len(list_network) - 1)]
    return model, max_sensitivity




# list de liste jusqu'à une dataset
def from_list_to_dataset(dataset):
    """
    Convert a list of list into a usable dataset
    :param dataset: a list of list of float with the last column being the output
    :return: the input and the ouput (numpy.array)
    """
    X = []
    Y = []
    for line in dataset:
        l = []
        n = len(line)
    for i in range(n - 1):
        l.append(line[i])
    X.append(l)
    Y.append(line[n - 1])
    X = np.array(X)
    Y = np.array(Y)
    return X, Y


def padding(l):
    m = 0
    for element in l:
        if len(element) > m:
            m = len(element)
    for element in l:
        if len(element) < m:
            rest = m - len(element)
            for k in range(rest):
                element.append(0)
    return l

# mean d'une liste
def mean(l):
    sum = 0
    for element in l:
        sum += element
    return (sum/len(l))

def min_list(l):
    min = l[0]
    for element in l:
        if element < min:
            min = element
    return element

def max_list(l):
    max = l[0]
    for element in l:
        if element >max:
            max = element
    return max

In [2]:
import pandas


# ici cette fonction associe à un fund tous les stratégies qui lui correspond:
def preprocessing_fund(strategy): 
    """
    Create ,from a dataframe containing all the strategies of the fund, a dictionary associating its fund to all of
    its strategy :param strategy: name of the excel file (str) :return: a dictionary associating its fund to all of
    its strategy
    """
    strategy = pandas.read_excel(strategy)

    strategy.columns = ['Fonds', 'investstrategyid', 'lowunitticket', 'highunitticket',
                        'lowinvest', 'highinvest', 'currencyid', 'name', 'investstrategytypeid',
                        'createddate', 'modifieddate', 'structureid', 'investstrategyid.1',
                        'sectorid', 'Secteur', 'name.1', 'investstrategyid.2', 'localisationid',
                        'Géo', 'investstrategyid.3', 'regionid', 'Region', 'investstrategyid.4',
                        'operationtypeid', 'OperationType']

    # Take off all the non numerical attribute
    del strategy['createddate']
    del strategy['modifieddate']
    del strategy['Géo']
    del strategy['Region']
    del strategy['investstrategyid.3']
    del strategy['investstrategyid.4']
    del strategy['OperationType']
    del strategy['investstrategyid.1']
    del strategy['investstrategyid.2']
    del strategy['Secteur']
    del strategy['name.1']
    del strategy['investstrategytypeid']
    del strategy['name']
    del strategy['structureid']

    # Put everything under the same currency
    strategy['lowunitticket'] = strategy.apply(conversion_lowticket, axis=1)
    strategy['highunitticket'] = strategy.apply(conversion_highticket, axis=1)
    strategy['lowinvest'] = strategy.apply(conversion_lowinvest, axis=1)
    strategy['highinvest'] = strategy.apply(conversion_highinvest, axis=1)

    del strategy['currencyid']
    del strategy['operationtypeid']

    # Replace the missing value
    strategy['sectorid'] = strategy['sectorid'].transform(replace_missing_id)
    strategy['localisationid'] = strategy['localisationid'].transform(replace_missing_id)
    strategy['regionid'] = strategy['regionid'].transform(replace_missing_id)

    # Since a strategy can contain many sector, localisation and region
    # We're regrouping everything
    list_fund = strategy.Fonds.values
    list_strategyid = strategy.investstrategyid.values
    list_lowunitticket = strategy.lowunitticket.values
    list_highunitticket = strategy.highunitticket.values
    list_highinvest = strategy.highinvest.values
    list_lowinvest = strategy.lowinvest.values
    list_sectorid = strategy.sectorid.values
    list_localisationid = strategy.localisationid.values
    list_regionid = strategy.regionid.values

    fake_number_strat = len(list_strategyid)
    list_index_strat = []
    list_strat_done = []
    for i in range(fake_number_strat):
        strat = list_strategyid[i]
        if strat not in list_strat_done:
            l = []
            list_strat_done.append(strat)
            for j in range(fake_number_strat):
                if (list_strategyid[j] == strat):
                    l.append(j)
            list_index_strat.append(l)

    list_sectoridv1 = []
    list_localisationidv1 = []
    list_regionidv1 = []
    list_highunitticketv1 = []
    list_lowunitticketv1 = []
    list_highinvestv1 = []
    list_lowinvestv1 = []
    list_fundv1 = []

    for index in list_index_strat:
        sector = []
        localisation = []
        region = []
        highinvest = []
        fund = []
        for i in index:
            sector.append(list_sectorid[i])
            localisation.append(list_localisationid[i])
            region.append(list_regionid[i])
            if len(highinvest) == 0:
                list_highunitticketv1.append(list_highunitticket[i])
                list_lowunitticketv1.append(list_lowunitticket[i])
                list_lowinvestv1.append(list_lowinvest[i])
                list_highinvestv1.append(list_highinvest[i])
                fund.append(list_fund[i])
        list_sectoridv1.append(sector)
        list_localisationidv1.append(localisation)
        list_regionidv1.append(region)
        list_fundv1.append(fund)

    for i in range(len(list_sectoridv1)):
        list_sectoridv1[i] = list(set(list_sectoridv1[i]))

    for i in range(len(list_localisationidv1)):
        list_localisationidv1[i] = list(set(list_localisationidv1[i]))

    for i in range(len(list_regionidv1)):
        list_regionidv1[i] = list(set(list_regionidv1[i]))

    list_sectorid = padding(list_sectoridv1)
    list_localisationid = padding(list_localisationidv1)
    list_regionid = padding(list_regionidv1)
    list_fund = []
    for fund in list_fundv1:
        for element in fund:
            if element not in list_fund:
                list_fund.append(element)

    list_highunitticket = list_highunitticketv1
    list_lowunitticket = list_lowunitticketv1
    list_highinvest = list_highinvestv1
    list_lowinvest = list_lowinvestv1

    list_fundv1 = list(set(list_fund))
    strategy_dictionnary = {}
    for fund in list_fundv1:
        strategy_dictionnary[fund] = []
    for i in range(len(list_fund)):
        fund = list_fund[i]
        strategy_dictionnary[fund].append([list_sectorid[i], list_localisationid[i], list_regionid[i],
                                      list_highunitticket[i],
                                      list_lowunitticket[i], list_highinvest[i], list_lowinvest[i]])
    return strategy_dictionnary


In [3]:

import pandas
import json

# ici cette fonction crée un dictionnaire qui associe à chaque company ses valeurs clefs aprés les avoir transformé
# en des données catégories.
def preprocessing_company(excel, name_dictionary):

    # Importing the file
    companies = pandas.read_excel(excel)

    # Taking off the missing values
    companies = take_off_missing_value(companies, True, 30, True)

    # Take off all the non numerical and useless attribute
    del companies['createddate']
    del companies['modifieddate']
    del companies['street']
    del companies['city']
    del companies['zipcode']
    del companies['secteur']
    del companies['pays']
    del companies['region']
    del companies['secondarysectorid']
    del companies['deactivateddate']
    del companies['latitude']
    del companies['longitude']
    del companies['birthdate']
    del companies['tcambegindate']
    del companies['tcamenddate']

    # We may use this one later
    del companies['website']

    # Renaming the columns

    companies.columns = ['companyid', 'name', 'localisationid', 'sectorid', 'tcam', 'regionid',
                         'isdeactivated', 'keyword', 'ca', 'ebitda']

    # Convert the keyword into a categorical value
    list_keyword = companies.keyword.values
    list_name_company = companies.name.values
    categorical_keyword = from_values_to_categorical(list_keyword)[0]

    # Creating a dictionary of keyword for each company

    dictionary_company_keyword = {}
    for i in range(len(list_name_company)):
        name_company = list_name_company[i]
        dictionary_company_keyword[name_company] = []
    max_numb_keyword = 0
    for i in range(len(list_name_company)):
        name_company = list_name_company[i]
        dictionary_company_keyword[name_company].append(categorical_keyword[i])
        if len(dictionary_company_keyword[name_company]) > max_numb_keyword:
            max_numb_keyword = len(dictionary_company_keyword[name_company])

    # All companies won't have the same number of keyword ,so we add zero
    for company in dictionary_company_keyword.keys():
        if len(dictionary_company_keyword[company]) < max_numb_keyword:
            n_keys = max_numb_keyword - len(dictionary_company_keyword[company])
            for j in range(n_keys):
                dictionary_company_keyword[company].append(0)

    # We take off the companies appearing twice
    companies.drop_duplicates(subset="name", keep='first', inplace=True)

    def associate_key_to_value(key):
        return dictionary_company_keyword[key]

    companies['keyword'] = companies['name'].apply(associate_key_to_value)

    # Converting non numerical attributes into categorical values
    companies['isdeactivated'] = from_values_to_categorical(companies.isdeactivated.values)[0]
    del companies['companyid']

    # Converting the datafram into a dictionnary
    company_dictionnary = companies.set_index('name').T.to_dict('list')

    # Registering the dictionary
    with open(name_dictionary, "w") as outfile:
        json.dump(company_dictionnary, outfile)
    return company_dictionnary


In [4]:

import pandas
import json

# ici une fonction qui donne un dictionnaire associant à chaque company ses investors 
def preprocess_portfolio(excel, dictionary_company_portfolio_name, dictionary_fund_deal_name):
    """

    :param excel: name of excel file of the portfolio (str)
    :param dictionary_company_portfolio_name: name of the
    dictionary containing each investors of a given company(str)
    :param dictionary_fund_deal_name: name of the
    dictionary containing each deal of a given fund (str)
    :return: a dictionary containing each investors of a given
    company and dictionary containing each deal of a given fund (dic)
    """
    # Importing the file
    portfolios = pandas.read_excel(excel)

    # Take off all the non numerical and useless attributes

    del portfolios['portfoliocompanyid']
    del portfolios['companyid']
    del portfolios['entryvalorisation']
    del portfolios['modifieddate']
    del portfolios['exitvalorisation']
    del portfolios['structureid']
    del portfolios['operationtype']
    del portfolios['createddate']
    del portfolios['builduprate']
    del portfolios['isowned']
    del portfolios['operationtypeid']
    del portfolios['issearchaddon']

    # Replacing the missing value
    portfolios['exitdate'] = portfolios['exitdate'].transform(replacemissingexitsdate)
    portfolios['entrydate'] = portfolios['entrydate'].transform(replacemissingentrysdate)

    # Convert the date into numbers
    portfolios['entrydate'] = from_date_to_number(portfolios.entrydate.values, ' %Y-%m-%d %H:%M:%S ')
    portfolios['exitdate'] = from_date_to_number(portfolios.exitdate.values, ' %Y-%m-%d %H:%M:%S ')

    # We're creating a dictionary containing each investors of a given company

    company = portfolios.company.values
    funds = portfolios.fonds.values
    unique_company = list(set(company))
    unique_funds = list(set(funds))

    dictionary_company_portfolio = {}
    for i in range(len(unique_company)):
        name_company = unique_company[i]
        dictionary_company_portfolio[name_company] = []

    for i in range(len(company)):
        name_company = company[i]
        dictionary_company_portfolio[name_company].append(funds[i])

    # We're creating a dictionary containing each deal of a given fund

    list_entry_date = portfolios.entrydate.values
    list_exit_date = portfolios.exitdate.values

    dictionary_fund_deal = {}
    for i in range(len(unique_funds)):
        name_fund = unique_funds[i]
        dictionary_fund_deal[name_fund] = []

    for j in range(len(company)):
        deal = [company[j], list_entry_date[j], list_exit_date[j]]
        name_fund = funds[j]
        dictionary_fund_deal[name_fund].append(deal)

    # Registering the dictionaries
    with open(dictionary_company_portfolio_name, "w") as outfile:
        json.dump(dictionary_company_portfolio, outfile)

    with open(dictionary_fund_deal_name, "w") as outfile:
        json.dump(dictionary_fund_deal, outfile)
    return dictionary_fund_deal, dictionary_company_portfolio


In [5]:
def history(company, fund, dictionary_fund_deal, company_dictionary):
    main_attributes = company_dictionary[company]
    list_previous_company = [x[0] for x in dictionary_fund_deal[fund]]
    list_last_deal = [x[2] for x in dictionary_fund_deal[fund]]
    last_deal = max(list_last_deal)
    nb_invest = len(dictionary_fund_deal[fund])
    nb_local = 0
    nb_region = 0
    nb_sector = 0
    list_ca = []
    nb_comp_with_info = 0
    for comp in list_previous_company:
        if comp in company_dictionary.keys():
            nb_comp_with_info += 1
            attribut = company_dictionary[comp]
            if attribut[0] == main_attributes[0]:
                nb_local += 1
            if attribut[1] == main_attributes[1]:
                nb_sector += 1
            if attribut[3] == main_attributes[3]:
                nb_region += 1
            list_ca.append(attribut[6])
    if nb_comp_with_info > 0:
        prop_local = nb_local / nb_invest
        prop_sector = nb_sector / nb_invest
        prop_region = nb_region / nb_invest
        histo = [last_deal, nb_local, nb_sector, nb_region, max_list(list_ca),
                 min_list(list_ca), mean(list_ca)]
    else:
        histo = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    return histo

In [6]:
def matching(strategy_dictionnary, company_dictionnary, dictionary_company_portfolio, dictionary_fund_deal_name):
    list_company = dictionary_company_portfolio.keys()
    list_fund = dictionary_fund_deal_name.keys()
    list_for_dataframe = []
    print('Matching fund and company ')

    # For each company
    for company in list_company:

        # For each fund
        for fund in list_fund:

            # If we have a deal involving this company
            if company in company_dictionnary.keys():

                # And a strategy related to this fund
                if fund in strategy_dictionnary.keys():
                    if fund in dictionary_company_portfolio[company]:
                        isinvestor = True
                    else :
                        isinvestor = False
                    list_strategy = strategy_dictionnary[fund]
                    line = [company, fund]
                    number_strategy = len(list_strategy)
                    number_matching_strategy = 0
                    histo = history(company, fund, dictionary_fund_deal_name, company_dictionnary)
                    for element in histo:
                        line.append(element)

                    # For each strategy of the fund
                    for strategy in list_strategy:
                        matching = True
                            # for attributes in strategy:
                            # if (type(attributes) == float) or (type(attributes) == np.int64) or (
                            # type(attributes) == int) or (type(attributes) == np.float64):
                            # line.append(attributes)

                            # else:
                            # for aspect in attributes:
                            # line.append(aspect)
                            # for attributes in company_dictionnary[company]:
                            # if (type(attributes)) == float or (type(attributes) == int):
                            # line.append(attributes)
                            # else:
                            # for aspect in attributes:
                            # line.append(aspect)

                            # If the the sector from the company corresponds to one in the strategy
                        if (company_dictionnary[company][1] not in strategy[0]) and (1 not in strategy[0]):
                            matching = False

                            # If the localisation
                        if company_dictionnary[company][0] not in strategy[1]:
                            matching = False

                            # If the region
                        if company_dictionnary[company][3] not in strategy[2]:
                            matching = False
                        if matching:
                            number_matching_strategy += 1
                    score_matching = number_matching_strategy / number_strategy
                    line.append(number_matching_strategy)
                    if isinvestor:
                        line.append(1)
                    else:
                        line.append(0)
                    del line[0]
                    del line[0]
                    list_for_dataframe.append(line)
    return list_for_dataframe

In [7]:
# We first create a dictionary where we associate each fund to all of its strategy
strategy_dictionary = preprocessing_fund('strategy.xlsx')

# We create a dictionary where we associate each company to all of its attribute
company_dictionary = preprocessing_company("companies.xlsx", "company_dictionnary.json")

# We now create 2 different dictionary
# 1 associating each fund to all the companies he has already invested in
# 1 associating each company to all the companies that have already invested in it

dictionary_fund_deal_name, dictionary_company_portfolio = preprocess_portfolio("portfolioscompanies.xlsx",
                                                                               'dictionary_company_portfolio.json',
                                                                               'dictionary_fund_deal_name.json')

# We now create a list containing all the attribute of the strategy of the fund, all the attribute of the companies
# and if the fund invested in the company or not (quite enormous)
list_for_dataframe = matching(strategy_dictionary, company_dictionary, dictionary_company_portfolio,
                              dictionary_fund_deal_name)



Matching fund and company 


In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from csv import reader
import sklearn.decomposition
import matplotlib.pyplot as plt
import keras.backend as K

In [9]:
def from_list_to_dataset(dataset):
  "Convert a list into an usable dataset"
  X = []
  Y = []
  for line in dataset :
    l = []
    n = len(line)
    for i in range(n-1):
      l.append(float(line[i]))
    X.append(np.asarray(l).astype(np.float32))
    Y.append(float(line[n-1]))
  return X,Y

In [10]:
def sensitivity(y_true, y_pred): 
  "Compute the sensitivity of the classifier"
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  return true_positives / (possible_positives + K.epsilon())

In [16]:
dataset = list_for_dataframe

In [13]:
len(X[0])

8

In [17]:
X,Y = from_list_to_dataset(dataset)
n = len(X[0])
# Delete the elements in which data are missing
for i in range(len(X)) :
  element = X[i]
  if len(element)!= n:
    print(i)
    del X[i]
    del Y[i]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
11100121
11100186
11100198
11100202
11100269
11100358
11100361
11100365
11100449
11100475
11100484
11100502
11100509
11100547
11100581
11100614
11100664
11100700
11100702
11100731
11100744
11100748
11100749
11100770
11100772
11100812
11100862
11100885
11100912
11100914
11100954
11100957
11100964
11100972
11100974
11100978
11100985
11100991
11101056
11101068
11101072
11101139
11101228
11101231
11101235
11101319
11101345
11101354
11101372
11101379
11101417
11101451
11101484
11101534
11101570
11101572
11101601
11101614
11101618
11101619
11101640
11101642
11101682
11101732
11101755
11101782
11101784
11101824
11101827
11101834
11101842
11101844
11101848
11101855
11101861
11101926
11101938
11101942
11102009
11102098
11102101
11102105
11102189
11102215
11102224
11102242
11102249
11102287
11102321
11102354
11102404
11102440
11102442
11102471
11102484
11102488
11102489
11102510
11102512
11102552
11102602
11102625
1110

IndexError: ignored

In [18]:
# Convert everything into a numpy array
X = np.asarray(X).astype("float32")
Y = np.asarray(Y).astype("float32")

In [19]:
# Split the dataset in 2 : 1 for the training, one for the test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42,stratify = Y)

In [20]:
# Creating the neural network
# Naming it 
multi_layer_reg_model = tf.keras.Sequential(name='multi_layer_regularization')
# Adding the input layer : as much neuron as input
multi_layer_reg_model.add(tf.keras.layers.Input(X.shape[1]))
# Adding the hidden layer 
multi_layer_reg_model.add(tf.keras.layers.Dense(10, activation='softmax', kernel_regularizer=tf.keras.regularizers.l1(1), name='hidden'))
# Adding the ouput layer : as much neurons as potential output
multi_layer_reg_model.add(tf.keras.layers.Dense(2, activation='softmax', name='output'))
# Defining the metric to use : 
# we choose sensititivity since the we want our neural network to classify the one who already invest as investors
# But the one who didn't invest yet may still be potential investor 
multi_layer_reg_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[sensitivity])
# Recap of the the model parameters
multi_layer_reg_model.summary()

Model: "multi_layer_regularization"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 10)                90        
_________________________________________________________________
output (Dense)               (None, 2)                 22        
Total params: 112
Trainable params: 112
Non-trainable params: 0
_________________________________________________________________


In [21]:
# The training phase will stop if the sensitivity don't improve of more than 1e-2 in 2 epochs
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1e-2)
# Save only the best model
checkpoint = tf.keras.callbacks.ModelCheckpoint('multi_layer_best.h5', monitor='sensitivity', verbose=1 ,mode ='max')

In [ ]:
# Split the dataset in 2 : 1 for the training, one for the test
list_sensitivity=[]
for i in range(100):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state= i ,stratify = Y)
# training the model 
  multi_layer_reg_train = multi_layer_reg_model.fit(X_train, y_train ,callbacks=[earlystop,checkpoint], epochs=10, batch_size=2000, validation_data=(X_test,y_test))
  loss, acc = multi_layer_reg_model.evaluate(X_test, y_test)
  list_sensitivity.append(acc)

Epoch 1/10
4207/4207 [==============================] - 22s 3ms/step - loss: 1.1223 - sensitivity: 0.9629 - val_loss: 0.0292 - val_sensitivity: 0.9572

Epoch 00001: saving model to multi_layer_best.h5
Epoch 2/10
4207/4207 [==============================] - 11s 3ms/step - loss: 0.0238 - sensitivity: 0.9608 - val_loss: 0.0237 - val_sensitivity: 0.9572

Epoch 00002: saving model to multi_layer_best.h5
Epoch 3/10
4207/4207 [==============================] - 11s 3ms/step - loss: 0.0217 - sensitivity: 0.9596 - val_loss: 0.0214 - val_sensitivity: 0.9572

Epoch 00003: saving model to multi_layer_best.h5
Epoch 4/10
4207/4207 [==============================] - 11s 3ms/step - loss: 0.0216 - sensitivity: 0.9627 - val_loss: 0.0223 - val_sensitivity: 0.9572

Epoch 00004: saving model to multi_layer_best.h5
Epoch 5/10
4207/4207 [==============================] - 11s 3ms/step - loss: 0.0216 - sensitivity: 0.9670 - val_loss: 0.0216 - val_sensitivity: 0.9572

Epoch 00005: saving model to multi_layer_bes

In [ ]:
fig, (loss_ax, acc_ax) = plt.subplots(1, 2, figsize=(20,7))

loss_ax.set_title('Loss')
loss_ax.plot(multi_layer_reg_train.history['loss'], '-r', label='Train')

acc_ax.set_title('Accuracy')
acc_ax.plot(multi_layer_reg_train.history['sensitivity'], '-r', label='Train')

plt.legend(loc=4)
plt.show()

In [ ]:
def predict_investor(company,company_dictionary,dictionnary_fund_deal_name,strategy_dictionary,dictionary_company_portfolio, model):
  list_potential_investor = []
  company_attributes = company_dictionary[company]
  for fund in dictionary_fund_deal_name.keys():
    vector_for_predictions = []
    
    # If we have a strategy related to this fund
    if fund in strategy_dictionary.keys():
      
      # if the fund didn't already invest in the company
      #if fund not in dictionary_company_portfolio[company]:
      list_strategy = strategy_dictionary[fund]
      line = [company, fund]
      number_strategy = len(list_strategy)
      number_matching_strategy = 0
      histo = history(company, fund, dictionary_fund_deal_name, company_dictionary)
      for element in histo:
          line.append(element)

        # For each strategy of the fund
      for strategy in list_strategy:
        matching = True
        
      # If the the sector from the company corresponds to one in the strategy
        if (company_dictionary[company][1] not in strategy[0]) and (1 not in strategy[0]):
          matching = False

        # If the localisation
        if company_dictionary[company][0] not in strategy[1]:
          matching = False

        # If the region
        if company_dictionary[company][3] not in strategy[2]:
          matching = False
        
        # We consider that the company fit to the strategy 
        if matching:
          number_matching_strategy += 1
        
        # We want to know the proportion of strategy into which the company fit in 
      score_matching = number_matching_strategy / number_strategy
      line.append(score_matching)
        
        # We delete the name of the fund the company
      del line[0]
      del line[0]
        
        # We predict if the fund may be a potential investor with our model
      vector_for_predictions = np.array(line).reshape(1,11)
      predictions = model.predict(vector_for_predictions)
      value_predictions = np.argmax(predictions, axis=1)[0]
      print(value_predictions)
        
        # If it is we add it to the list
      if value_predictions ==1 :
        list_potential_investor.append(fund)
  return list_potential_investor 





In [ ]:
dictionnary_company = company_dictionary
company = 'Multiplast-Dulary'
model = multi_layer_reg_model

In [ ]:
loss, acc = multi_layer_reg_model.evaluate(X_test, y_test)
print('Accuracy: {}'.format(acc))

In [ ]:
predict_investor(company,company_dictionary,dictionary_fund_deal_name,strategy_dictionary,dictionary_company_portfolio, model)